# Rajasthan Green Cover Quick Analysis
Quick notebook for testing vegetation monitoring

In [ ]:
import sys
sys.path.append('../backend')

import ee
from vegetation_monitor import VegetationMonitor
from config import DISTRICTS
import pandas as pd

## Initialize Earth Engine

In [ ]:
# Initialize (make sure you've run 'earthengine authenticate' first)
try:
    ee.Initialize()
    print("✓ Earth Engine initialized successfully")
except Exception as e:
    print(f"✗ Error: {e}")
    print("Run 'earthengine authenticate' in terminal first")

## Analyze Jodhpur District

In [ ]:
monitor = VegetationMonitor()

# Analyze Jodhpur
jodhpur_results = monitor.analyze_district('Jodhpur')
monitor.print_summary(jodhpur_results)

## Analyze Bikaner District

In [ ]:
# Analyze Bikaner
bikaner_results = monitor.analyze_district('Bikaner')
monitor.print_summary(bikaner_results)

## Visualize Results

In [ ]:
import matplotlib.pyplot as plt

# Compare districts
districts = ['Jodhpur', 'Bikaner']
ndvi_values = [
    jodhpur_results['current_week']['ndvi_mean'],
    bikaner_results['current_week']['ndvi_mean']
]
changes = [
    jodhpur_results['change']['ndvi_change_mean'],
    bikaner_results['change']['ndvi_change_mean']
]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Current NDVI
ax1.bar(districts, ndvi_values, color=['green', 'darkgreen'])
ax1.set_ylabel('NDVI')
ax1.set_title('Current Week NDVI by District')
ax1.set_ylim([0, 1])
ax1.axhline(y=0.4, color='orange', linestyle='--', label='Moderate Vegetation')
ax1.legend()

# Week-over-week change
colors = ['red' if c < 0 else 'green' for c in changes]
ax2.bar(districts, changes, color=colors)
ax2.set_ylabel('NDVI Change')
ax2.set_title('Week-over-Week Change')
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

## Export Results

In [ ]:
import json
from datetime import datetime

# Prepare results for export
export_data = {
    'Jodhpur': {k: v for k, v in jodhpur_results.items() if k != 'images'},
    'Bikaner': {k: v for k, v in bikaner_results.items() if k != 'images'}
}

# Save to file
filename = f'../data/analysis_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
with open(filename, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"Results saved to {filename}")